## Confirmed Cases

In [20]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [21]:
filename = "covidComplete.csv"
df = pd.read_csv(filename)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Datetime   129 non-null    object
 1   Confirmed  129 non-null    int64 
 2   Recovered  129 non-null    int64 
 3   Deceased   129 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 4.2+ KB
None


In [22]:
df.head()

,Datetime,Confirmed,Recovered,Deceased
0,2020-01-29 00:00:00,0,0,0
1,2020-01-30 00:00:00,1,0,0
2,2020-01-31 00:00:00,1,0,0
3,2020-02-01 00:00:00,1,0,0
4,2020-02-02 00:00:00,2,0,0


In [23]:
df.set_axis(df['Datetime'], inplace=True)
df.drop(columns=['Recovered','Deceased'], inplace=True)

In [24]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

trace1 = go.Scatter(x=df.Datetime,
                    y=df.Confirmed,
                    name = "plotly example",
                    line = dict(color = 'blue'),
                    opacity = 0.4)

layout = dict(title='plotly example',)

fig = dict(data=[trace1], layout=layout)
iplot(fig)

In [25]:
close_data = df['Confirmed'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.90
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Datetime'][:split]
date_test = df['Datetime'][split:]

print(len(close_train))
print(len(close_test))

116
13


In [26]:
look_back = 5


train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=10)

In [27]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(40,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 15
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/15
6/6 [==============================] - 1s 144ms/step - loss: 1140921227.1509
Epoch 2/15
6/6 [==============================] - 0s 17ms/step - loss: 1066577886.1659
Epoch 3/15
6/6 [==============================] - 0s 17ms/step - loss: 1007550851.0027
Epoch 4/15
6/6 [==============================] - 0s 21ms/step - loss: 901730781.5291
Epoch 5/15
6/6 [==============================] - 0s 34ms/step - loss: 841509733.3342
Epoch 6/15
6/6 [==============================] - 0s 36ms/step - loss: 785800610.5423
Epoch 7/15
6/6 [==============================] - 0s 21ms/step - loss: 623999026.2398
Epoch 8/15
6/6 [==============================] - 0s 28ms/step - loss: 575301613.4320
Epoch 9/15
6/6 [==============================] - 0s 29ms/step - loss: 500008445.9031
Epoch 10/15
6/6 [==============================] - 0s 30ms/step - loss: 452516398.2962
Epoch 11/15
6/6 [==============================] - 0s 31ms/step - loss: 336588110.5792
Epoch 12/15
6/6 [==============================]

In [28]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Covid cases",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [29]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Datetime'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 25
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [30]:
# prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
forecast = forecast.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Confirmed Cases",
    xaxis = {'title' : "Dates"},
    yaxis = {'title' : "Confirmed"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

## Recovered Cases

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator


filename = "covidComplete.csv"
df = pd.read_csv(filename)
print(df.info())



Using TensorFlow backend.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Datetime   129 non-null    object
 1   Confirmed  129 non-null    int64 
 2   Recovered  129 non-null    int64 
 3   Deceased   129 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 4.2+ KB
None


In [2]:
df.head()

,Datetime,Confirmed,Recovered,Deceased
0,2020-01-29 00:00:00,0,0,0
1,2020-01-30 00:00:00,1,0,0
2,2020-01-31 00:00:00,1,0,0
3,2020-02-01 00:00:00,1,0,0
4,2020-02-02 00:00:00,2,0,0


In [3]:
df.set_axis(df['Datetime'], inplace=True)
df.drop(columns=['Confirmed','Deceased'], inplace=True)

In [4]:
df.head()

,Datetime,Recovered
Datetime,,
2020-01-29 00:00:00,2020-01-29 00:00:00,0
2020-01-30 00:00:00,2020-01-30 00:00:00,0
2020-01-31 00:00:00,2020-01-31 00:00:00,0
2020-02-01 00:00:00,2020-02-01 00:00:00,0
2020-02-02 00:00:00,2020-02-02 00:00:00,0


In [5]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

trace1 = go.Scatter(x=df.Datetime,
                    y=df.Recovered,
                    name = "plotly example",
                    line = dict(color = 'blue'),
                    opacity = 0.4)

layout = dict(title='plotly example',)

fig = dict(data=[trace1], layout=layout)
iplot(fig)



In [6]:
close_data = df['Recovered'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.90
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Datetime'][:split]
date_test = df['Datetime'][split:]

print(len(close_train))
print(len(close_test))

look_back = 5


train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=10)


116
13


In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(40,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 15
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)



Epoch 1/15
6/6 [==============================] - 1s 237ms/step - loss: 135113170.5552
Epoch 2/15
6/6 [==============================] - 0s 21ms/step - loss: 147376854.0222
Epoch 3/15
6/6 [==============================] - 0s 17ms/step - loss: 127581406.2271
Epoch 4/15
6/6 [==============================] - 0s 19ms/step - loss: 116200407.5185
Epoch 5/15
6/6 [==============================] - 0s 19ms/step - loss: 95528841.3273
Epoch 6/15
6/6 [==============================] - 0s 37ms/step - loss: 87615262.5651
Epoch 7/15
6/6 [==============================] - 0s 19ms/step - loss: 71502076.3749
Epoch 8/15
6/6 [==============================] - 0s 21ms/step - loss: 67880458.8677
Epoch 9/15
6/6 [==============================] - 0s 30ms/step - loss: 59479415.7715
Epoch 10/15
6/6 [==============================] - 0s 19ms/step - loss: 53379897.5764
Epoch 11/15
6/6 [==============================] - 0s 18ms/step - loss: 48982039.1586
Epoch 12/15
6/6 [==============================] - 0s 20m

In [8]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Datetime'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Covid cases",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()
num_prediction = 25
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)


In [9]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Datetime'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 25
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [10]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
forecast = forecast.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Recovered cases vs Dates",
    xaxis = {'title' : "Dates"},
    yaxis = {'title' : "Recovered Cases"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()


## Deceased Cases

In [11]:
filename = "covidComplete.csv"
df = pd.read_csv(filename)
print(df.info())

df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Datetime   129 non-null    object
 1   Confirmed  129 non-null    int64 
 2   Recovered  129 non-null    int64 
 3   Deceased   129 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 4.2+ KB
None


,Datetime,Confirmed,Recovered,Deceased
0,2020-01-29 00:00:00,0,0,0
1,2020-01-30 00:00:00,1,0,0
2,2020-01-31 00:00:00,1,0,0
3,2020-02-01 00:00:00,1,0,0
4,2020-02-02 00:00:00,2,0,0


In [12]:
df.set_axis(df['Datetime'], inplace=True)
df.drop(columns=['Recovered','Confirmed'], inplace=True)

In [13]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

trace1 = go.Scatter(x=df.Datetime,
                    y=df.Deceased,
                    name = "plotly example",
                    line = dict(color = 'blue'),
                    opacity = 0.4)

layout = dict(title='plotly example',)

fig = dict(data=[trace1], layout=layout)
iplot(fig)

In [14]:
close_data = df['Deceased'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Datetime'][:split]
date_test = df['Datetime'][split:]

print(len(close_train))
print(len(close_test))

103
26


In [15]:
look_back = 8


train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=10)

In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(40,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 15
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/15
5/5 [==============================] - 1s 166ms/step - loss: 457314.0539
Epoch 2/15
5/5 [==============================] - 0s 24ms/step - loss: 455214.7841
Epoch 3/15
5/5 [==============================] - 0s 24ms/step - loss: 451771.9622
Epoch 4/15
5/5 [==============================] - 0s 26ms/step - loss: 449541.9748
Epoch 5/15
5/5 [==============================] - 0s 25ms/step - loss: 439197.6033
Epoch 6/15
5/5 [==============================] - 0s 24ms/step - loss: 263319.9388
Epoch 7/15
5/5 [==============================] - 0s 32ms/step - loss: 218807.4659
Epoch 8/15
5/5 [==============================] - 0s 25ms/step - loss: 198514.1330
Epoch 9/15
5/5 [==============================] - 0s 24ms/step - loss: 136707.0024
Epoch 10/15
5/5 [==============================] - 0s 26ms/step - loss: 121792.8213
Epoch 11/15
5/5 [==============================] - 0s 36ms/step - loss: 75346.4877
Epoch 12/15
5/5 [==============================] - 0s 28ms/step - loss: 25836.4692
Ep

In [17]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Datetime'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Covid cases",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()
num_prediction = 25
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)


In [18]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Datetime'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 25
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [19]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
forecast = forecast.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Death cases vs Dates",
    xaxis = {'title' : "Dates"},
    yaxis = {'title' : "Deaths"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()
